# A Mongolian News Language Model from Scratch

Big thanks to [Florian Leuerer](https://github.com/floleuerer) for his ULMFiT notebooks for German ([repo link](https://github.com/floleuerer/fastai_ulmfit_german)). They helped significantly!

We will be using the smaller [Eduge dataset](https://github.com/tugstugi/mongolian-nlp/blob/master/datasets/eduge.csv.gz) provided by Bolorsoft LLC for this language model. This first language model is more a proof of concept that will be scaled up in later notebooks.

## Library Installs and Imports

We will need several libraries to get started. Each should be installed and upgraded to ensure it works. This notebook was created in February 2020 and should work with the following major verisons:

- Fast.ai version 2.x
- Fastcore version: 1.x
- Pandas version: 1.x
- Numpy version: 1.x

In [ ]:
!pip install -Uqq fastai --upgrade
!pip install -Uqq fastcore --upgrade
!pip install -Uqq pandas==1.1.0

In [ ]:
import fastai
import fastcore
print('Fast.ai version:', fastai.__version__)
print('Fastcore version:', fastcore.__version__)

Fast.ai version: 2.2.7
Fastcore version: 1.3.19


In [ ]:
from fastai.text.all import *
from fastai.callback.progress import CSVLogger
import pandas as pd
import numpy as np
print('Pandas version:', pd.__version__)
print('numpy version:', np.__version__)

Pandas version: 1.1.0
numpy version: 1.19.5


We will be using Google Drive to store our files. Please change the directory for the `cd` command to wherever your notebook is located. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/DataScience/NLP/01_eduge_classification'

/content/drive/MyDrive/DataScience/NLP/Eduge Classification


## Create Data Loader for Language Model

We will be using data in a Pandas Dataframe to build our language model. However, you could also use txt files. 

In [ ]:
df = pd.read_csv('data/news.csv')

In [ ]:
df = df.rename(columns={'news':'text'})

In [ ]:
df.head(2)

,text,label
0,"Киноны кадраас «Юрийн галавын үе» кино прокатад тавигдсныхаа дараах эхний амралтын өдрөөр хамгийн их орлого оллоо гэж Business Insider хэвлэл мэдээлэв. Хойд Америкт л гэхэд эхний амралтын өдрүүдэд 204,6 сая доллар цуглуулж чадлаа. Гадаад орнуудын үзүүлэлт нь 307,2 сая ам.доллар байв. Нийт кассын орлого 511,8 сая ам.доллар болжээ. Энэхүү үзүүлэлт нь Universal кино компанийн дээд амжилт төдийгүй дэлхийн кино аж үйлдвэрийн томоохон үсрэлт боллоо. Үүнээс өмнө амралтын эхний өдрүүдэд 500 сая долларын босго давсан их мөнгө цуглуулж байсан түүх байхгүй. «Гарри Поттер ба үхлийн тахил: II хэсэг» 48...",урлаг соёл
1,"“Universal” кино компани Жадд Апатоугийн найруулж, “Lonely Island” гэдэг хошин шогийн хамтлагийнхны гол дүрүүдэд нь тоглож байгаа киног 2016 оны зургадугаар сарын 3-нд дэлгэцнээ гаргахаар төлөвлөсөн тухай The Hollywood Reporter мэдээлж байна. Шинэ киноны нэрийг одоохондоо өгөөгүй гэнэ. Кинонд инээдмийн жүжигчид Энди Сэмберг, Йорма Таккоме, Акива Шаффер нар тогложээ. Сүүлийн жилүүдэд гарч ирсэн баримтат киноны элэглэл нь шинэ кино болох ёстой. Энд Жастин Бибер, Кэти Перри нарын элэглэл хамгийн түрүүнд дүрслэгдэх аж. “Lonely Island” хошин шогийн хамтлагийнхан “Saturday Night Live” гэдэгт нэв...",урлаг соёл


We will be using the ColSplitter, which expects a column called is_valid (with a boolean) to generate the validation set. We will randomly assign 10% to the validation set.

In [ ]:
np.random.seed(42)

In [ ]:
df['is_valid'] = np.random.choice([0,1], size=len(df), p=[0.9, 0.1])

We will set the batch size to 128 and the `seq_len` to 80. These can be changed depending on your GPU. 

The `is_lm` is set to `True` because we are training a language model. We are using a `get_x` but not a `get_y` because our target variable will be generated for us by the fastai library. 

In [ ]:
bs = 128

In [ ]:
dblock = DataBlock(blocks=TextBlock.from_df('text', is_lm=True),
                   get_x=ColReader('text'),
                   splitter=ColSplitter())

dls = dblock.dataloaders(df, bs=bs, seq_len=80)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Now you can see the `text_` column, which is our target variable. Our language model will be attempting to predict the next token in the sequence. 

In [ ]:
dls.show_batch(max_n=2)

,text,text_
0,"xxbos гэрэл зургийг xxunk сайтаас авав . бразилийн рио - де - жанейрод болох зуны спортын их наадмын сонгомол барилдаанд оролцох чөлөөтийн бөхчүүдийг сонгон шалгаруулах цуврал тэмцээний хугацаа , зохион байгуулах улс , олгох эрхийн тоог хэрхэн тогтоох тухай албан мэдээллийг дэлхийн бөхийн төрлүүдийн нэгдсэн холбоо саяхан гаргажээ . олимпийн наадамд зодоглох эрхийг дөрвөн шатлалтай сонгон шалгаруулж олгохоор болжээ . ирэх жилийн олимпод сонгомол болон чөлөөт бөхөөр эрэгтэй , эмэгтэй тамирчид тус бүр зургаан жинд хүч үзнэ . xxunk xxunk","гэрэл зургийг xxunk сайтаас авав . бразилийн рио - де - жанейрод болох зуны спортын их наадмын сонгомол барилдаанд оролцох чөлөөтийн бөхчүүдийг сонгон шалгаруулах цуврал тэмцээний хугацаа , зохион байгуулах улс , олгох эрхийн тоог хэрхэн тогтоох тухай албан мэдээллийг дэлхийн бөхийн төрлүүдийн нэгдсэн холбоо саяхан гаргажээ . олимпийн наадамд зодоглох эрхийг дөрвөн шатлалтай сонгон шалгаруулж олгохоор болжээ . ирэх жилийн олимпод сонгомол болон чөлөөт бөхөөр эрэгтэй , эмэгтэй тамирчид тус бүр зургаан жинд хүч үзнэ . xxunk xxunk эрэгтэй"
1,". энэхүү мэдэгдлийг нийслэлийн бүх их дээд сургуулиуд , мсүт , коллежийн захиргаа , оюутны өөрөө удирдах байгууллагад хүргүүлж дараа дараагийн арга хэмжээг авахаар төлөвлөж буй юм байна . э.энх - эрдэнэ xxbos нийслэлийн здтг , байгаль орчин , ногоон хөгжлийн яам , монголын хотуудын холбоотой хамтран “ ногоон хот ” семинарыг зохион байгууллаа . семинарт нийслэлийн засаг даргын хот байгуулалт , хөрөнгө оруулалтын асуудал хариуцсан орлогч ▁ с.очирбат , байгаль орчин , ногоон хөгжлийн яам , монголын хотуудын холбооны","энэхүү мэдэгдлийг нийслэлийн бүх их дээд сургуулиуд , мсүт , коллежийн захиргаа , оюутны өөрөө удирдах байгууллагад хүргүүлж дараа дараагийн арга хэмжээг авахаар төлөвлөж буй юм байна . э.энх - эрдэнэ xxbos нийслэлийн здтг , байгаль орчин , ногоон хөгжлийн яам , монголын хотуудын холбоотой хамтран “ ногоон хот ” семинарыг зохион байгууллаа . семинарт нийслэлийн засаг даргын хот байгуулалт , хөрөнгө оруулалтын асуудал хариуцсан орлогч ▁ с.очирбат , байгаль орчин , ногоон хөгжлийн яам , монголын хотуудын холбооны төлөөлөгч"


Our training set is 2563 sequences of 128 tokens. Our validation hold out set is 290 sequences. 

In [ ]:
len(dls.train), len(dls.valid)

(2573, 280)

Here we can move to the language model training, or if the training is already compplete, move to below to test the model.

## Training The Language Model

In [ ]:
notebook_path = Path('')

We will be using the AWD_LSTM architecture for our language model. We set the `pretrained` option to `False` as we will not be using a pretrained language model (that's what we are doing!).

In [ ]:
learn = language_model_learner(dls, AWD_LSTM, drop_mult=0.5, pretrained=False, 
                               metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
learn.path = notebook_path.absolute()

Here we set our learning rate. Ideally we would find our ideal learning rate for our data. 

**Note:** *room for improvement*

In [ ]:
lr = 1e-2
lr *= bs/48  # Scale learning rate by batch size

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, lr, moms=(0.8, 0.7, 0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,5.100392,5.032722,0.242005,153.349792,32:32
1,4.789495,4.717944,0.259048,111.937889,32:50
2,4.623714,4.534988,0.271817,93.222435,32:26
3,4.454101,4.397218,0.282083,81.224609,32:26
4,4.347083,4.277759,0.292213,72.078705,32:24
5,4.232885,4.152819,0.302854,63.613083,32:23
6,4.067704,4.028792,0.314193,56.192986,32:17
7,3.969931,3.916042,0.325851,50.201347,32:32
8,3.840867,3.843464,0.334433,46.686901,32:12
9,3.718772,3.828775,0.336651,46.006165,32:23


We acheived a 33% accuracy, which isn't bad for such a small dataset. Now that we have it trained we will save two files:

- The model itself, called `mn_eduge.pth`. 
- The vocabulary the model contains, called `mn_eduge_voacb.pkl`.
- The encoder, called '`mn_encoder.pth`.

Both are saved in the `notebook_path/models` directory (Fast.ai by default saves into `models` for wherever your learner path is.

In [ ]:
learn.to_fp32().save('mn_eduge_lm', with_opt=False)

Path('models/models/mn_eduge.pth')

In [ ]:
with open('models/mn_eduge_vocab.pkl', 'wb') as f:
      pickle.dump(learn.dls.vocab, f)

In [ ]:
learn.save_encoder('mn_eduge_lm_encoder')

## Test out the model

We won't expect the results to be very good, but we can at least see if the language model is learning reasonably decent. 

In [ ]:
lm_fns = ['models/mn_eduge_lm', 'models/mn_eduge_vocab']

In [ ]:
learn = language_model_learner(dls, AWD_LSTM, drop_mult=0.5, pretrained=True, 
                               pretrained_fnames=lm_fns, model_dir='', metrics=[accuracy, Perplexity()]).to_fp16()

/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/autocast_mode.py:114: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")


In [ ]:
TEXT = "Барилга угсралт их засварын ажил"

In [ ]:
preds = []
N_WORDS = 50
N_SENTENCES = 1
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [ ]:
print("\n".join(preds))

барилга угсралт их засварын ажил удаашралтай байгаа нь барилгын материалын үйлдвэрлэлийн компанид шалгалт хийжээ . энэ үеэр барилгын материалын үйлдвэр , барилгын материал нийлүүлэх , импортлоход тавих хяналт , шалгалтыг хийж буй аж . уг арга хэмжээг бнхау - аас монгол улсад суугаа онц бөгөөд бүрэн эрхт элчин сайд в.в.самойленко нээж , олон улсын валютын
